# Generate global Voronoi shapes for `adm1` regions
- Here, `adm1` includes `adm0` for which there are no `adm1` regions

### Todo:
- comment heavily on Voronoi-original-shape logic

INPUTS
- `sset.PATH_GADM_ADM1`

OUTPUTS
- `sset.PATH_GADM_ADM0_VORONOI`
- `sset.PATH_GADM_ADM1_VORONOI`

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
import pygeos
from shapely.geometry import Polygon

from sliiders import spatial as pv
from sliiders import settings as sset

pv.filter_spatial_warnings()

In [ ]:
full_regions = gpd.read_parquet(sset.PATH_GADM_ADM1)

region_polys = full_regions.explode(index_parts=False)

In [ ]:
# This has been tested with XYZ coordinates so cannot guarantee performance on more complex shapefiles
assert (
    pygeos.count_coordinates(pygeos.from_shapely(region_polys["geometry"]))
    < sset.MAX_VORONOI_COMPLEXITY
)

In [ ]:
gridded_gdf, all_oc = pv.grid_gdf(region_polys)

pts_df = pv.polys_to_vor_pts(region_polys, all_oc)

In [ ]:
vor_gdf = pv.get_spherical_voronoi_gdf(pts_df)

vor_gdf.plot()

In [ ]:
vor_shapes = pygeos.from_shapely(vor_gdf["geometry"])
all_gridded = pygeos.from_shapely(gridded_gdf["geometry"])

tree = pygeos.STRtree(all_gridded)

vor_ix, existing = tree.query_bulk(vor_shapes, "intersects")

gridded_uid = np.take(gridded_gdf["UID"].to_numpy(), existing)
vor_uid = np.take(vor_gdf["UID"].to_numpy(), vor_ix)

In [ ]:
vor_gdf["calculated"] = pv.remove_already_attributed_land_from_vor(
    existing, vor_shapes, vor_ix, gridded_uid, vor_uid, all_gridded
)

vor_gdf["calculated"].plot()

In [ ]:
vor_gdf = vor_gdf.drop(columns=["geometry"]).rename(columns={"calculated": "geometry"})

full_regions = pd.merge(
    full_regions,
    vor_gdf.rename(columns={"geometry": "calculated"}),
    left_on="UID",
    right_on="UID",
    how="left",
)

full_regions["calculated"] = full_regions["calculated"].fillna(Polygon())

full_regions.head()

In [ ]:
full_regions["combined"] = full_regions["geometry"].union(full_regions["calculated"])

full_regions["combined"].plot(figsize=(20, 20))

In [ ]:
out = full_regions[full_regions["UID"].notnull()][["UID", "combined"]].rename(
    columns={"combined": "geometry"}
)
out = gpd.GeoDataFrame(out)

out["geometry"] = out["geometry"].apply(pv.grab_polygons)
out["geometry"] = out["geometry"].apply(pv.strip_line_interiors)

out.plot()

In [ ]:
out = pv.fill_in_gaps(out)

### Save `adm1` and `adm0` files

In [2]:
out = gpd.read_parquet(sset.PATH_GADM_ADM1_VORONOI)

In [4]:
out["ISO"] = out["UID"].str.split(".").str[0]
out.to_parquet(sset.PATH_GADM_ADM1_VORONOI, index=False)

In [ ]:
adm0 = out.dissolve("ISO", as_index=False).drop(columns=["UID"])
adm0.to_parquet(sset.PATH_GADM_ADM0_VORONOI, index=False)